In [ ]:
#all_flag
#hide
from medtop.core import *
from medtop.internal import *
from medtop.nlp_helpers import *
from medtop.preprocessing import *
from nbdev.showdoc import *
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#hide
# wide screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# MedTop

> Extracting topics from reflective medical writings

Website: https://cctrbic.github.io/medtop/  
![CI](https://github.com/cctrbic/medtop/workflows/CI/badge.svg)

## Install

`pip install medtop`

## How to use

Download NLTK data to your machine using `python -m nltk.downloader all`

Import documents from a text file containing a list of all documents  
**QUESTION: I can't see anywhere we are actually using `my_docs_pos` or `my_docs_loc`. Can we remove these?**

In [ ]:
#TODO: some documents aren't loading properly
path_to_file_list = 'data/2019.03.11_DevCorpus/file_list.txt'
data, doc_df = import_docs(path_to_file_list)

You can review the imported documents manually by writing the raw sentences to disk.

In [ ]:
sentences_to_disk(data)

Create the TF-IDF matrix. `path_to_corpus_file_list` can be a list of just input files (Local) or a list of all files in the corpus (Global)

In [ ]:
path_to_seed_topics_file_list = 'data/2019.03.12_SEED_TOPICS/FILELIST.txt'
tfidf, dictionary = create_tfidf(path_to_seed_topics_file_list, doc_df)

Size of TF-IDF: 14 Docs, 860 Tokens


Extract the most expressive phrase from each sentence.  

**Need a better understanding of `include_input_in_tfidf` and `token_averages = get_term_max(tdm)`. They don't seem to be representative of their names.**  

**Also, I think this is sloppy how I'm appending a column in place. Probably want to refactor.**

In [ ]:
#TODO: Remove None phrase here? Probably add a print statement notifying user
data = append_phrase_column(data, dictionary.token2id, tfidf, include_input_in_tfidf = False)

In [ ]:
# Filter out None phrase records
data = data[data.phrase.notnull()]

### Create sentence vectors
**TODO: add explanations**
- TF-IDF
- SVD
- UMAP
- Pre-trained Word2Vec embeddings
- Create Word2Vec embeddings from input corpus

In [ ]:
data = append_vec_column("tfidf", data, dictionary = dictionary, tfidf = tfidf)
# data = append_vec_column("svd", data, dictionary = dictionary, tfidf = tfidf)
# data = append_vec_column("umap", data, dictionary = dictionary, tfidf = tfidf)
# data = append_vec_column("pretrained", data, path_to_w2v_bin_file = "data/pubmed2018_w2v_200D.bin")
# data = append_vec_column("local", data, doc_df = doc_df)

Filter the sentences, removing any that contain zero "top phrases"  
**QUESTION: After filtering here, we never again use the filtered records in my_docs, could we filter in place to make this cleaner?**  
**QUESTION: Is there a reason we're storing `just_phrase_ids` as a concatenated string instead of a tuple? We can always convert to string when writing to a file, but it makes for messier logic than necessary.**

### Clustering

In [ ]:
data = assign_clusters(data, method = "hac")
data.head()

### Visualization
All visualization uses cosine similarity. **Why?**

In [ ]:
# dist = pairwise_distances(just_phrase_vecs, metric='cosine')

**QUESTION: Should visualization be three separate methods or a single method with optional parameters? There are a few other similar instances in the code where we could do multiple top-level methods or a single caller with kwargs**

In [ ]:
# # TODO: This is not displaying correctly in the README
# visualize_umap(just_phrase_ids, cluster_assignments, just_phrase_text, dist)
# # visualize_mds(just_phrase_ids, cluster_assignments, just_phrase_text, dist)
# # visualize_svd(just_phrase_ids, cluster_assignments, just_phrase_text, dist)

**QUESTION: The variables here are named `cluster_topics`, but it seems like they're actual lists of phrases in each cluster. Need clarification**

In [ ]:
# main_cluster_topics = get_cluster_topics(cluster_assignments, just_phrase_ids, my_docs)

### Write output to files

In [ ]:
# write_output_to_disk(main_cluster_topics, just_phrase_ids, cluster_assignments, raw_sentences, doc_top_phrases, file_list, file_name = "output/TopicClusterResults.txt")
# evaluate(just_phrase_ids, cluster_assignments, just_phrase_text)

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted Core.ipynb.
Converted index.ipynb.
Converted internal.ipynb.
Converted nlp_helpers.ipynb.
Converted preprocessing.ipynb.
Converted Sandbox.ipynb.
